# Ray Tune - A Deeper Dive Using MNIST with PyTorch

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

A [previous notebook](02-Understanding-Hyperparameter-Tuning.ipynb) explained the concept of hyperparameter tuning/optimization (HPO) and walked through the basics of using [Ray Tune](https://ray.readthedocs.io/en/latest/tune.html), and another [notebook on Tune and Sklearn](03-Ray-Tune-with-Sklearn.ipynb) showed Tune's drop-in replacements for HPO.

Now we'll use another example to explore more of the Tune API features. We'll use the [MNIST](http://yann.lecun.com/exdb/mnist/) of hand-written digits and train a [PyTorch](https://pytorch.org/) model to recognize them.

In [1]:
import os 
from torchvision import datasets, transforms
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from filelock import FileLock

## PyTorch Hyperparameter Tuning

Our example will closely follow the code in the [PyTorch MNIST example](https://github.com/pytorch/examples/blob/master/mnist/main.py). However, we will create an even simpler model than the one in the example, although you could try that model and compare its predictions.

Let's start by defining a few global variables for epoch and test sizes. Also define a data location.

In [2]:
EPOCH_SIZE = 512
TEST_SIZE = 256

DATA_ROOT = '../data/mnist'

The following class defines a convolutional neural network.

> **Tip:** Most of these code definitions can be found in `mnist.py`, too.

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

After creating that network, we can now create our data loaders for training and test data. These are just plain [PyTorch `DataLoaders`](https://pytorch.org/docs/1.1.0/data.html?highlight=dataloader#torch.utils.data.DataLoader) with two additions:

1. A `FileLock` is added to ensure that only one process downloads the data on each machine, just in case we have multiple workers per machine in our Ray cluster.
2. The root directory for the data can be specified and it will be created if it doesn't exist.

Otherwise, this code is identical to the [PyTorch example version](https://github.com/pytorch/examples/blob/master/mnist/main.py#L101).

In [4]:
def get_data_loaders():
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    # We add FileLock here because multiple workers on the same machine coulde try 
    # download the data. This would cause overwrites, since DataLoader is not threadsafe.
    # You wouldn't need this for single-process training.
    lock_file = f'{DATA_ROOT}/data.lock'
    import os
    if not os.path.exists(DATA_ROOT):
        os.makedirs(DATA_ROOT)
        
    with FileLock(os.path.expanduser(lock_file)):
        train_loader = torch.utils.data.DataLoader(
            datasets.MNIST(DATA_ROOT, train=True, download=True, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)

        test_loader = torch.utils.data.DataLoader(
            datasets.MNIST(DATA_ROOT, train=False, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
    return train_loader, test_loader

Now we define our training and test functions. While the arguments are a bit switched up from the original PyTorch tutorial, the difference is inconsequential. The arguments are an optimizer, a model, the training data loader, and our device. Then we train the model.

In [5]:
def train(model, optimizer, train_loader, device=torch.device("cpu")):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

Similarly for our test model, we define a basic _average correct prediction_ metric that we will track. We could add more metrics, but we'll keep it simple.

In [6]:
def test(model, data_loader, device=torch.device("cpu")):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total

Finally, we create a wrapper function for this particular model. In doing so all we need to do is specify the configuration for the model that we would like to train and the function will do the rest:

1. Retrieve the data with the loaders returned by `get_data_loaders()`
2. Create the `ConvNet` model
3. Optimize the model using _stochastic gradient descent_.

In [18]:
def train_mnist(config):
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        print(f"accuracy: {acc}")

### Single-Node Hyperparameter Tuning

Let's show what we might do if we performed hyperparameter tuning on a single machine. We would have to enumerate all the possibilities and either train them serially or use something like multiprocessing to train them in parallel. That setup takes a little bit of work so people often decide to train them serially, which is easiest, but requires the most time.

This is what we might do.

In [19]:
import itertools
conf = {
    "lr": [0.001, 0.01, 0.1],
    "momentum": [0.001, 0.01, 0.1, 0.9]
}

combinations = list(itertools.product(*conf.values()))
print(len(combinations))
combinations

12


[(0.001, 0.001),
 (0.001, 0.01),
 (0.001, 0.1),
 (0.001, 0.9),
 (0.01, 0.001),
 (0.01, 0.01),
 (0.01, 0.1),
 (0.01, 0.9),
 (0.1, 0.001),
 (0.1, 0.01),
 (0.1, 0.1),
 (0.1, 0.9)]

In [20]:
for lr, momentum in combinations:
    train_mnist({"lr":lr, "momentum":momentum})
    break # we'll stop this after one run and just use it for illustrative purposes

accuracy: 0.109375
accuracy: 0.103125
accuracy: 0.0875
accuracy: 0.109375
accuracy: 0.090625
accuracy: 0.121875
accuracy: 0.140625
accuracy: 0.1375
accuracy: 0.1125
accuracy: 0.134375


### Distributed Hyperparameter Tuning with Ray Tune

Ray Tune makes it trivial to move this code from a single node to multiple nodes. Let's see how to use the code we've written with Ray Tune.

First, we set up Ray as before.

In [21]:
import ray
from ray import tune

In [22]:
ray.init(ignore_reinit_error=True)

2021-11-18 17:07:59,459	INFO services.py:1272 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:6379',
 'object_store_address': '/tmp/ray/session_2021-11-18_17-07-58_034492_32575/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-11-18_17-07-58_034492_32575/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-11-18_17-07-58_034492_32575',
 'metrics_export_port': 59372,
 'node_id': '8d4ff2b14997852f26aceb4aafd94b02734862ede2a7ce72f154dd43'}

The first change is we'll perform a strict `grid_search` on our hyperparameters, like we used in the previous lesson. Our hyperparameters are the learning rate, `lr`, and the `momentum`.

In [23]:
config = {
    "lr": tune.grid_search([0.001, 0.01, 0.1]),
    "momentum": tune.grid_search([0.001, 0.01, 0.1, 0.9])
}

Next we modify our trainable, `train_mnist`, to use Tune's "reporting" logger:

In [24]:
def train_mnist(config):
    from ray.tune import report
    train_loader, test_loader = get_data_loaders()
    model = ConvNet()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)
        # This sends the score to Tune.
        report(mean_accuracy=acc)

That's all that we need to change in order for Ray Tune to be able to parallelize our different hyperparameter combinations. 

When we execute a hyperparameter sweep, we perform an **experiment**. Each distinct combination of our different hyperparameters constitutes a single **trial**.

## Tune's Functional vs. Class API

In the above previous lesson, we used the **functional API**. This API is most convenient for quickly setting up experiments, but it provides less overall flexbility compared to the **class API** [`tune.Trainable`](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#tune-trainable).

We'll try both, starting with the functional API.

We add a stopping criterion, `stop={"training_iteration": 20}`, so this will go reasonably quickly, while still producing good results. Consider removing this condition if you don't mind waiting longer and you want optimal results.

**Note**: Unlike the functional API, in which you the trainable can call a `tune.report()`, the class API method `cls.step()` can only return a value.

In [25]:
%%time
analysis_func = tune.run(train_mnist, config=config, stop={"training_iteration": 20},
                         verbose=1)

2021-11-18 17:28:25,580	INFO tune.py:630 -- Total run time: 8.00 seconds (7.82 seconds for the tuning loop).


CPU times: user 816 ms, sys: 226 ms, total: 1.04 s
Wall time: 8.04 s


In [26]:
print("Best config: ", analysis_func.get_best_config(metric="mean_accuracy", mode="max"))

Best config:  {'lr': 0.1, 'momentum': 0.9}


In [27]:
analysis_func.dataframe().sort_values('mean_accuracy', ascending=False).head()

,mean_accuracy,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
11,0.871875,0.196317,False,NaN,NaN,10,f917a_00011,312cb61ff52d487cb93a3fa98f9a66e5,2021-11-18_17-28-25,1637285305,5.613645,38586,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.613645,0,10,0.10,0.900,/Users/jules/ray_results/train_mnist_2021-11-1...
5,0.868750,0.293517,False,NaN,NaN,10,f917a_00005,a664bbee77744b26a623f7cb64381d38,2021-11-18_17-28-24,1637285304,5.128907,38580,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.128907,0,10,0.10,0.010,/Users/jules/ray_results/train_mnist_2021-11-1...
8,0.856250,0.220953,False,NaN,NaN,10,f917a_00008,7233108e53a4480c8a4f200d1623e720,2021-11-18_17-28-25,1637285305,5.384853,38579,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.384853,0,10,0.10,0.100,/Users/jules/ray_results/train_mnist_2021-11-1...
10,0.825000,0.191259,False,NaN,NaN,10,f917a_00010,0b2111815e2043fdb4eed645d926d5df,2021-11-18_17-28-25,1637285305,5.588994,38585,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.588994,0,10,0.01,0.900,/Users/jules/ray_results/train_mnist_2021-11-1...
1,0.765625,0.211650,False,NaN,NaN,10,f917a_00001,4eeb10c311e64daf975e8e4a6069869e,2021-11-18_17-28-25,1637285305,5.471587,38581,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.471587,0,10,0.01,0.001,/Users/jules/ray_results/train_mnist_2021-11-1...


In [28]:
analysis_func.dataframe()[['mean_accuracy', 'config/lr', 'config/momentum']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/lr,config/momentum
11,0.871875,0.100,0.900
5,0.868750,0.100,0.010
8,0.856250,0.100,0.100
10,0.825000,0.010,0.900
1,0.765625,0.010,0.001
2,0.765625,0.100,0.001
4,0.762500,0.010,0.010
7,0.740625,0.010,0.100
9,0.568750,0.001,0.900
0,0.162500,0.001,0.001


How long did it take? We'll compare this value with a different training run in the next lesson.

In [29]:
stats = analysis_func.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

   0.23 seconds,    0.00 minutes


### Use Tune's Trainable Class API

As a subclass of tune.Trainable, Tune will create a Trainable object on a separate process (using the [Ray Actor API](https://docs.ray.io/en/latest/actors.html#actor-guide)).

 * setup function is invoked once training starts.
 * step is invoked multiple times. Each time, the Trainable object executes one logical iteration of training in the tuning process, which may include one or more iterations of actual training.


In [30]:
class TrainMNIST(tune.Trainable):
    def setup(self, config):
        self.config = config
        self.train_loader, self.test_loader = get_data_loaders()
        self.model = ConvNet()
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.config["lr"])
    
    def step(self):
        train(self.model, self.optimizer, self.train_loader)
        acc = test(self.model, self.test_loader)
        return {"mean_accuracy": acc}

In [32]:
%%time
analysis = tune.run(
    TrainMNIST, 
    config=config,
    stop={"training_iteration": 20},
    verbose=1
)

2021-11-18 17:31:17,099	INFO tune.py:630 -- Total run time: 13.34 seconds (13.19 seconds for the tuning loop).


CPU times: user 2.12 s, sys: 408 ms, total: 2.53 s
Wall time: 13.4 s


In [33]:
print("Best config: ", analysis.get_best_config(metric="mean_accuracy", mode="max"))

Best config:  {'lr': 0.1, 'momentum': 0.01}


In [34]:
# Get a dataframe for analyzing trial results.
df = analysis.dataframe()
df.head()

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
0,0.178125,True,NaN,NaN,20,5c245_00000,fc205119dcad4e839036bf304d6aa913,2021-11-18_17-31-14,1637285474,0.322850,5.809877,41087,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,5.809877,0,20,0.001,0.001,/Users/jules/ray_results/TrainMNIST_2021-11-18...
1,0.806250,True,NaN,NaN,20,5c245_00001,b288986fd75b4e35913cb3ea72793063,2021-11-18_17-31-15,1637285475,0.333669,6.193870,41089,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.193870,0,20,0.010,0.001,/Users/jules/ray_results/TrainMNIST_2021-11-18...
2,0.881250,True,NaN,NaN,20,5c245_00002,9b8be8887e9042128d1832cc7a189ada,2021-11-18_17-31-16,1637285476,0.293854,6.257735,41098,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.257735,0,20,0.100,0.001,/Users/jules/ray_results/TrainMNIST_2021-11-18...
3,0.278125,True,NaN,NaN,20,5c245_00003,1c8e317eea7e4a5699c97645b4ec10b7,2021-11-18_17-31-16,1637285476,0.301631,6.257378,41097,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.257378,0,20,0.001,0.010,/Users/jules/ray_results/TrainMNIST_2021-11-18...
4,0.846875,True,NaN,NaN,20,5c245_00004,f0fef02145dc495ca99eecb920d0cdcb,2021-11-18_17-31-16,1637285476,0.290808,6.191537,41095,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.191537,0,20,0.010,0.010,/Users/jules/ray_results/TrainMNIST_2021-11-18...


In [35]:
analysis.dataframe().sort_values('mean_accuracy', ascending=False).head()

,mean_accuracy,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,config/lr,config/momentum,logdir
5,0.906250,True,NaN,NaN,20,5c245_00005,70088eb49d164715ae1ce73514157093,2021-11-18_17-31-16,1637285476,0.298787,6.239424,41096,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.239424,0,20,0.10,0.010,/Users/jules/ray_results/TrainMNIST_2021-11-18...
11,0.893750,True,NaN,NaN,20,5c245_00011,fba4ca6e8a554b64a622b825664847ad,2021-11-18_17-31-15,1637285475,0.303682,6.064892,41092,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.064892,0,20,0.10,0.900,/Users/jules/ray_results/TrainMNIST_2021-11-18...
2,0.881250,True,NaN,NaN,20,5c245_00002,9b8be8887e9042128d1832cc7a189ada,2021-11-18_17-31-16,1637285476,0.293854,6.257735,41098,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.257735,0,20,0.10,0.001,/Users/jules/ray_results/TrainMNIST_2021-11-18...
8,0.871875,True,NaN,NaN,20,5c245_00008,04409843cf9b4369938680c1e4a2c643,2021-11-18_17-31-16,1637285476,0.297223,6.288576,41090,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.288576,0,20,0.10,0.100,/Users/jules/ray_results/TrainMNIST_2021-11-18...
7,0.850000,True,NaN,NaN,20,5c245_00007,0cedc1127026438f9090cc1cadb7c56a,2021-11-18_17-31-16,1637285476,0.230868,6.280406,41088,Juless-MacBook-Pro-16-inch-2019,127.0.0.1,6.280406,0,20,0.01,0.100,/Users/jules/ray_results/TrainMNIST_2021-11-18...


It's easier to see what we want if project out the interesting columns:

In [36]:
analysis.dataframe()[['mean_accuracy', 'config/lr', 'config/momentum']].sort_values('mean_accuracy', ascending=False)

,mean_accuracy,config/lr,config/momentum
5,0.906250,0.100,0.010
11,0.893750,0.100,0.900
2,0.881250,0.100,0.001
8,0.871875,0.100,0.100
7,0.850000,0.010,0.100
4,0.846875,0.010,0.010
10,0.818750,0.010,0.900
1,0.806250,0.010,0.001
6,0.300000,0.001,0.100
3,0.278125,0.001,0.010


How long did it take? We'll compare this value with a different training run in the next lesson.

In [37]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

  10.87 seconds,    0.18 minutes


The next lesson will explore optimization algorithms that speed up HPO.

In [ ]:
ray.shutdown()  # "Undo ray.init()".